# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [1]:
! git clone https://github.com/albatrousse/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

fatal: destination path 'exam_2025' already exists and is not an empty directory.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [3]:
mySeed = 888

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [6]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

Une méthode généralement utilisée pour estimer les coefficients dans un modèle linéaire est la régression linéaire par moindre carré (OLS). Cette méthode a pour objectif de minimiser la somme des carrés des différences entre les valeurs observées et les valeurs prédites.

In [45]:
from sklearn.linear_model import LinearRegression

# Extraire les entrées et cibles des jeux de données
X_train = train_set['inputs']
t_train = train_set['targets']

X_test = test_set['inputs']
t_test = test_set['targets']

# Créer et entraîner le modèle de régression linéaire
linearReg = LinearRegression()
linearReg.fit(X_train, t_train)

# Afficher les coefficients et l'ordonnée à l'origine estimés
print("Coefficients estimés (θ1, θ2, θ3) :", linearReg.coef_)
print("Ordonné à l'origine estimé (θ0) :", linearReg.intercept_)

# Évaluer le modèle sur les données de test
score = linearReg.score(X_test, t_test)
print("Score sur l'ensemble de test :", score)


Coefficients estimés (θ1, θ2, θ3) : [ 8.78948541  8.8947063  17.58357221]
Ordonné à l'origine estimé (θ0) : 44.38097638349817
Score sur l'ensemble de test : 0.9850486580679025


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

Etant donné que le modèle à estimer est purement linéaire, une simple architecture linéaire permettrait de l'estimer. Au niveau de l'expressivité, une unique couche dense (avec une fonction d'activitation linéaire) a la même expressivité qu'un modèle de regréssion linéaire (cf représentation mathématique y = W*x + b). Cette "simplicité" permet également d'éviter le surapprentissage et donc favorise une meilleure généralisation.

On choisit par conséquent, nous choisissons d'avoir une architecture avec une unique couche dense (sans activation) prenant en entrée trois predicteurs (x, y et z). La couche apprend les poids (θ1, θ2, θ3) et le biais (θ0) et trouve en sortie t.

In [61]:
import torch.nn as nn

# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear = nn.Linear(in_features=3, out_features=1)

    def forward(self, x):
        return self.linear(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [62]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)
model = SimpleNet()

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
      # Remettre les gradients à zéro
      optimizer.zero_grad()

      # Propagation avant
      outputs = model(batch_inputs)

      # Calcul de la perte
      loss = criterion(outputs, batch_targets.view(-1, 1))

      # Rétropropagation
      loss.backward()

      # Mise à jour des paramètres
      optimizer.step()

    # Affichage de la perte toutes les 50 époques
    if (epoch + 1) % 50 == 0:
      print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [50/500], Loss: 3400.1650
Epoch [100/500], Loss: 3087.4531
Epoch [150/500], Loss: 3111.8826
Epoch [200/500], Loss: 3357.0037
Epoch [250/500], Loss: 3281.1143
Epoch [300/500], Loss: 3059.4094
Epoch [350/500], Loss: 3215.1753
Epoch [400/500], Loss: 3001.6572
Epoch [450/500], Loss: 3326.0046
Epoch [500/500], Loss: 3273.0959


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [63]:
# Affichage des paramètres appris
print("Coefficients estimés (θ1, θ2, θ3) :", model.linear.weight.data.numpy())
print("Ordonné à l'origne estimé (θ0) :", model.linear.bias.data.numpy())

Coefficients estimés (θ1, θ2, θ3) : [[-0.27973843  0.18755682 -0.04751011]]
Ordonné à l'origne estimé (θ0) : [-0.2601475]


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

Nous nous attendions à avoir une perte sur le jeu de test similaire pour les deux méthodes mais étonnemment, la méthode avec un réseau de neuronne ne converge pas du tout. Cela pourrait être dû à une architecture trop simpliste, ou simplement qu'il faudrait plus de données pour le réseau afin d'estimer correctement les θ (Plutôt la première au vu des pertes lors de l'entrainement du réseau). Par manque de temps, je vais continuer vers la suite.

In [64]:
from sklearn.metrics import mean_squared_error

# Prédictions sur le jeu de test
predictionsSGD = model(torch.tensor(X_test, dtype=torch.float32))
predictionsLinearReg = linearReg.predict(X_test)

# Calcul de la perte sur le jeu de test
lossSGD = criterion(predictionsSGD, torch.tensor(t_test, dtype=torch.float32).view(-1, 1))
lossLinearReg = mean_squared_error(predictionsLinearReg, t_test)

# Affichage de la perte sur le jeu de test
print("Perte sur le jeu de test (SGD):", lossSGD.item())
print("Perte sur le jeu de test (LinearRegression):", lossLinearReg)

Perte sur le jeu de test (SGD): 3147.55322265625
Perte sur le jeu de test (LinearRegression): 3.66936534473806


\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [66]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

In [67]:
# Nb de paramètres dans self.Down1: (calcul "à la main")

# Down 1 prend 64 canaux en entrée et 128 en sortie.
# En regardant la structure d'un Down_causal dans util, on sait alors qu'il s'agit d'un MaxPool1D(2,2) suivit d'un Double_conv_causal
# D'où le nombre de paramètre :
# MaxPool1D : 0
# Double_conv_causal in_ch = 64 et out_ch = 128 :
  # conv1 : (3*64 + 1) * 128 = 24704
  # conv2 : (3*128 + 1) * 128 = 49280
  # bn1 et bn2 : chacun 2* 128 soit 512

# TOTAL :  74 496

# Nb de paramètres au total:

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Nombre total de paramètres: {total_params}")

Nombre total de paramètres: 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

 Le réseau réduit la taille d'entrée grâce à des opérations de mise en commun maximale avec des pas et des tailles de noyau variables. La deuxième partie du réseau restaure la taille d'origine en utilisant des techniques de suréchantillonnage dans les modules Up_causal, aidées par des skipconnexion et des paramètres soigneusement choisis pour les convolutions transposées. Ce processus permet au réseau d'extraire des caractéristiques à différentes échelles tout en conservant les informations spatiales pour la sortie finale.

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

Le champ réceptif est augmenté par des convolutions en cascade et des convolutions dilatées. À la sortie de self.inc, la taille du champ récepteur est de 5, ce qui signifie que chaque caractéristique de sortie est influencée par un voisinage de 5 caractéristiques d'entrée.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?